## Optimising Dream 11 Team Selection ##

### Section 1.1 Data Preparation :
1) Prepare a Score Vector (The mean of a Player's Score across 10 Matches is considered) <br>
2) Prepare Standard Deviation Vector( to capture variability or inconsistencies in player's scores). This is to be used in Mean Variance Optimisation

In [1]:
# !pip install gurobipy

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
#str = unicode(str, errors='ignore')
data_set_df=pd.read_csv(r'Players_Score_Names.csv', encoding='gbk')
data_set_df.head()

,Player Name,Player Type,Team,Retrospective Scores_10,Retrospective Scores_1,Retrospective Scores_2,Retrospective Scores_3,Retrospective Scores_4,Retrospective Scores_5,Retrospective Scores_6,Retrospective Scores_7,Retrospective Scores_8,Retrospective Scores_9,Price
0,Rohit Sharma,BAT,INDIA,278,374,387,372,380,477,481,357,474,477,11.5
1,James Anderson,BOWL,ENGLAND,220,300,275,147,121,72,226,141,125,547,9.5
2,Joe Root,BAT,ENGLAND,190,191,180,180,201,209,206,208,216,205,8.0
3,Virat Kohli,BAT,INDIA,454,251,442,436,133,437,442,250,294,461,11.0
4,Ravichandran Ashwin,BOWL,INDIA,424,228,378,21,417,365,429,425,420,119,9.0


#### Find mean and Standard Deviation of the scores of all 30 Players ###

In [2]:
score_df=data_set_df.iloc[0:,3:data_set_df.shape[1]-1] # Separate the score data from other details

In [3]:
sd_score=[np.std(score_df.iloc[i,:].values) for i in range(score_df.shape[0])] #Standard Deviation to measure inconsistency
scores=[np.mean(score_df.iloc[i,:].values) for i in range(score_df.shape[0])] #Score of each player

In [5]:
#########Other Parameters Set-up############

# Intialise a Player Type(Batsman,Bowler,Allrounder or Wicket Keeper) Matrix
ptype=np.zeros((4,30))

# Initialize a Team Matrix (The Dream 11 Team has to be formed from two competing Teams)
team=np.zeros((2,30))

# Fill in the Player and Team matrices with values from Dataset
for i in range(data_set_df.shape[0]):
    if(data_set_df.loc[i,'Player Type']=='BAT'):
        ptype[0,i]=1
    elif(data_set_df.loc[i,'Player Type']=='BOWL'):
        ptype[1,i]=1
    elif(data_set_df.loc[i,'Player Type']=='AR'):
        ptype[2,i]=1
    else:
        ptype[3,i]=1
    if(data_set_df.loc[i,'Team']=='INDIA'):
        team[0,i]=1
    else:
        team[1,i]=1

# Prepare a Budget Vector
price=data_set_df['Price'].values

# Prepare the lower bound and upper bound for Player Type . This is to be read as :
# The no. of batsman can be between 3 and 6 (lb[0] and ub[0] respectively)
# Similarly, the no. of bowlers can be between 3 and 6; No. of allrounders can be between 1 and 4; 
# and No of wicketkeeper can be between 1 and 4
lb=[3,3,1,1]
ub=[6,6,4,4]

# There can only be 1 Captain, 1 Vice Captain and the rest 9 players will be considered as normal players
player_role=[9,1,1]


N=data_set_df.shape[0] # No of Players available 
M=len(player_role) #No of Player Roles (N,VC,C)
K=ptype.shape[0] #No. of Player Types (4-BAT,BOWL,AR,WK)

# Risk_aversion captures the risk tolerance of a Dream 11 Player. 
#From a problem perspective, we aim to punish a player for his inconsistency. Risk_aversion captures the degree of punishment

risk_aversion=0

### Section 1.2 Stating the Objective Function and Constraints. Displaying the Output ###

In [6]:
def dream11_optimisation():
    data_set = Model()

    # Creat variables
    x=data_set.addVars(M, N, vtype=GRB.BINARY, name = "x")


    # Set objective : A Vice Captain has his score multiplied by 1.5 and A captain has his score multiplied by 2.
    # Additionally, a Player is punished for his inconsistency (A form of Mean Variance Optimisation, 
    # used mostly in financial portfolio management)
    data_set.setObjective( quicksum(scores[j]*x[0,j] for j in range(N))\
                  +quicksum(1.5*scores[j]*x[1,j] for j in range(N))\
                  +quicksum(2*scores[j]*x[2,j] for j in range(N))\
                  -risk_aversion*(quicksum(x[i,j]*sd_score[j] for i in range(M) for j in range(N))), GRB.MAXIMIZE)
    
    # The Player can either not be selected and if selected, can only take in one of the role : Either a Captain, Vice Captain or
    # an ordinary player
    data_set.addConstrs(( quicksum(x[i,j] for i in range(M)) <= 1 for j in range(N) )) ##equation 3
    
    # The below constraint ensures that there are 9 ordinary players, only 1 Vice Captain and only 1 Captain in the Team
    data_set.addConstrs(quicksum(x[i,j] for j in range(N)) ==player_role[i] for i in range(M))
    
    # Only 100 Points are available for Spending
    data_set.addConstr(quicksum(price[j]*x[i,j] for i in range(M) for j in range(N)) <=100) ##equation2
    
    # The no of Batsman,Bowler, Allrounder and WK has to be within the specified ranges
    data_set.addConstrs(quicksum(x[i,j]*ptype[k,j] for i in range(M) for j in range(N))<=ub[k] for k in range(K)) ##equation 6

    data_set.addConstrs(quicksum(x[i,j]*ptype[k,j] for i in range(M) for j in range(N))>=lb[k] for k in range(K)) ##equation 6

    # There can be a maximum of 7 players from one team
    data_set.addConstrs(quicksum(x[i,j]*team[k,j] for i in range(M) for j in range(N))<=7 for k in range(team.shape[0])) ##equation 5

    return data_set,x

In [ ]:
# risk_aversion can vary between 1 to 10: 
# A risk_aversion score of 0 means the risk taking apetite is huge. This Dream 11 Player doesn't take inconsistencies in 
# data_setket players performance into account while selecting his Playing 11. 
# 10 signifies that the risk-taking apetite of the Dream 11 player is almost negligible. He/She would prefer only consistent
# players in his/her playing 11
while (True):
    risk_aversion = int(input("Enter your risk tolerance : "))
    if(risk_aversion>10 or risk_aversion < 0 ):
        print("Invalid Input!!. Please Choose a tolerance limit between 0 and 10"+"(both inclusive)")
        continue
    else:
        print ('Risk Tolerance of the Dream-11 Player is',risk_aversion)
        # setup the model again
        dr_data_set,x = dream11_optimisation()
        

        
        # Solving the model
        dr_data_set.optimize()
        
        # Display the Output
    
        print('The Optimum Team for the upcoming match based on your risk tolerance is shown below:- \n')
        for v in dr_data_set.getVars():
            if(int(v.VarName[2])==2 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Captain of the Team is',data_set_df.loc[index,'Player Name'] + \
                  '('+ data_set_df.loc[index,'Player Type'] +')'+ ' from '+ data_set_df.loc[index,'Team'])
        for v in dr_data_set.getVars():
            if(int(v.VarName[2])==1 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Vice Captain of the Team is',data_set_df.loc[index,'Player Name'] + \
                      '('+ data_set_df.loc[index,'Player Type'] +')'+ ' from '+ data_set_df.loc[index,'Team']+'\n')
        print('Other Players:- \n')
        for v in dr_data_set.getVars():
            if(int(v.VarName[2])==0 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print(data_set_df.loc[index,'Player Name'] +'('+ data_set_df.loc[index,'Player Type'] +')'+ \
                      ' from '+ data_set_df.loc[index,'Team'])
        
        # Ask from User if he want's to select a team on other risk tolerance levels, apart from the one entered earlier
        choice = input("Enter (Y/N) if you wan't to see team combination for other risk_tolerance levels : \n")
        if(choice.upper()=='Y' ):
            continue
        else:
            print('No more Team Selections')
            break

Enter your risk tolerance : 1
Risk Tolerance of the Dream-11 Player is 1
Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 44 rows, 90 columns and 540 nonzeros
Model fingerprint: 0xd0de3115
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [9e-02, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 648.3685702
Presolve time: 0.09s
Presolved: 44 rows, 90 columns, 537 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 1.926521e+03, 59 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1926.52100    0    2  